# Long or short term planning

Having secured food supplies, reliable heat source, and efficient power lines, dr. Elena Vasilescu had solved her survival issues on the short term. Time to think long term! After doing the calculations, she realized that the current food supplies would last her only 90 sols. That was not too bad, however it was way too short for any rescue mission to reach her, and therefore futile. If only she could create some sort of food supply, she might be able to survive long enough until they come and get her. That is, unless she dies from something else first.

One of the objectives of the mission was to explore possibilities for growing food on Mars, and therefore they brought some potatoes with them to experiment with. She could decide between eating the potatoes, or trying to multiply them by planting them, which was an easy decision considering the circumstances.

The plant experimentation was supposed to happen inside a clean room that was set together with the hub. This would reduce the danger of contaminating the plants and increase their yield. But the only way to plant the potatoes in the clean room was using a robot present there.

<img src='../artwork/stranded/robot_planting_potatoes.png' width=60% />

To increase the yield of the plants, she had to create a plan and place the potatoes at specific coordinates on the field. This way, they should be neither too close (reducing yield), nor too far away from each other (wasting space). After she learned forward kinematics, she knew how to calculate the position of the robot for known joint coordinates. But how could she do the __inverse? How could she figure out the joint coordinates necessary so that the robot reaches a certain pose?

Should she start learning about _backwards_ kinematics? Nah... that sounds a little bit too silly.

# Forward versus inverse kinematics

In the previous laboratories, we analyzed the problem of calculating the pose and velocity of the end-effector of a robot arm under known joint coordinates and velocities. This is what we generally call __forward kinematics__, as it takes us from the variables that we control __directly__ on the robot (joint coordinates, velocities), into quantities that we can only control __indirectly__ (end-effector pose, velocity). Even though forward kinematics is useful, we usually start with a target end-effector pose as our objective, and we need to calculate the joint coordinates that result into the target pose. This is useful since we typically have a target pose that we need to reach with our robot. This is so we interact with an object (e.g., pick a box up), or so we get some sensor data (e.g., take a picture of an object/area).

## Theoretical considerations

As we've seen in previous laboratories, we usually represent pose using a $4 \times 4$ matrix, which consists of a $3 \times 3$ orientation matrix, and a $3 \times 1$ position vector. Even though the orientation matrix consists of 9 elements, not all of them are independent, as they must respect the following constrains:

* Each column of the matrix must correspond to a vector of unit length
* The three columns must represent three vectors perpendicular to each other

These constrains are translated into the following mathematical equations:

$$ \sqrt{X_x^2+X_y^2+X_z^2} = \sqrt{Y_x^2+Y_y^2+Y_z^2} = \sqrt{Z_x^2+Z_y^2+Z_z^2} = 1 $$

$$ \begin{bmatrix} X_x \\ X_y \\ X_z \end{bmatrix} \times \begin{bmatrix} Y_x \\ Y_y \\ Y_z \end{bmatrix} = \begin{bmatrix} Z_x \\ Z_y \\ Z_z \end{bmatrix} $$

$$ \begin{bmatrix} Y_x \\ Y_y \\ Y_z \end{bmatrix} \times \begin{bmatrix} Z_x \\ Z_y \\ Z_z \end{bmatrix} = \begin{bmatrix} X_x \\ X_y \\ X_z \end{bmatrix} $$

When respecting such constrains, it can be demonstrated, that only three elements of the orientation matrix are independent, while the rest are dependent on those. This can be interpreted as the three rotational degrees of freedom, each one around one axis of rotation. One way to represent these three rotations, is using the _Euler angles_, which represent a sequence of three rotations, each around one axis. The most common Euler sequence is the $X, Y, Z$ sequence, as a series of three rotations:

$$ Rz(\psi)Ry(\theta)Rx(\phi) = \begin{bmatrix}
  c_{\theta}c_{\psi} & c_{\psi}s_{\theta}s_{\phi} - c_{\phi}s_{\psi} & s_{\phi}s_{\psi} + c_{\phi}c_{\psi}s_{\theta} \\
  c_{\theta}s_{\psi} & c_{\phi}c_{\psi} + s_{\theta}s_{\phi}s_{\psi} & c_{\phi}s_{\theta}s_{\psi} - c_{\psi}s_{\phi} \\
  -s_{\theta} & c_{\theta}s_{\phi}  & c_{\theta}c_{\phi}
  \end{bmatrix} $$

this is often called the _Roll, Pitch, Yaw_ sequence, relating to aviation terminology.

To state things in more rigor, the __forward kinematics__ is a model that takes us from _Joint space_ into _Cartesian space_, and has the following form of a system of equations:

$$ Z = f(q) $$

Where $Z = [P_x, P_y, P_z, \phi, \theta, \psi]^T $ is a Cartesian pose expressed in terms of the $[P_x, P_y, P_z]^T $ position, and the orientation in terms of the $ [\phi, \theta, \psi]^T$ Euler angles. In the same formulation, the __inverse kinematics__ model, is a system of equations that take us from _Cartesian_ to _Joint_ space:

$$ q = g(Z) $$

Due to the highly non-linear nature of the forward kinematics (due to the appearance of trigonometric functions), this is not an easily invertible system of equations. Another complication is the fact that this set of non-linear equations might not have unique solutions for certain target poses. This is the case for instance in the 3 revolute joint robot in the figure below, which can achieve the same target Cartesian pose, with two different sets of joint coordinates.

<img src="../artwork/IKM/redundant.png" width=60% />

This issue is important when implementing trajectories, where continuity in the Cartesian trajectories might not necessarily result in continuities in the joint trajectories. Finally, another issue is that the inverse kinematics might have __zero__ solutions for some target poses.

## Calculating the Inverse Kinematics model

There are three main methodologies for solving the inverse kinematics problem:

* Analytical solutions
* Geometric solutions
* Numeric solutions

As with other fields of engineering, analytical and geometric solutions provide exact solutions, while numerical solutions are only approximations.

### Analytical solutions

To calculate an analytical solution of the inverse kinematics model, we need to first calculate the forward kinematics of the robot. If we want to express the orientation in terms of Euler angles, then we can equate the orientation sub-matrix of the forward kinematics with the orientation matrix resulting from the three Euler rotations:

$$ Rot_0^n = \begin{bmatrix}
  c_{\theta}c_{\psi} & c_{\psi}s_{\theta}s_{\phi} - c_{\phi}s_{\psi} & s_{\phi}s_{\psi} + c_{\phi}c_{\psi}s_{\theta} \\
  c_{\theta}s_{\psi} & c_{\phi}c_{\psi} + s_{\theta}s_{\phi}s_{\psi} & c_{\phi}s_{\theta}s_{\psi} - c_{\psi}s_{\phi} \\
  -s_{\theta} & c_{\theta}s_{\phi}  & c_{\theta}c_{\phi}
  \end{bmatrix} $$

which is a system of 9 equations, out of which 3 are independent (since all 9 elements depend only on the values of $\phi, \theta, \psi$). We should also equate the translation vector of the forward kinematics with the three position variables, resulting in three additional equations:

$$ Tr_0^n = \begin{bmatrix} P_x \\ P_y \\ P_z \end{bmatrix} $$

in total, we have 12 equations, however, there cannot be more than 6 independent equations (3 due to orientation, and 3 due to translation). Eventually, our variables for this system of equation are the joint coordinates $q = [q_1, q_2, \dots, q_n]^T$, and we can have maximum $n$ independent equations. Therefore, the number of independent equations is $min(6,n)$. We can write overally:

$$ T_0^n = GHM = \begin{bmatrix}
  c_{\theta}c_{\psi} & c_{\psi}s_{\theta}s_{\phi} - c_{\phi}s_{\psi} & s_{\phi}s_{\psi} + c_{\phi}c_{\psi}s_{\theta} & P_x\\
  c_{\theta}s_{\psi} & c_{\phi}c_{\psi} + s_{\theta}s_{\phi}s_{\psi} & c_{\phi}s_{\theta}s_{\psi} - c_{\psi}s_{\phi} & P_y \\
  -s_{\theta} & c_{\theta}s_{\phi}  & c_{\theta}c_{\phi} & P_z \\
  0 & 0 & 0 & 1
  \end{bmatrix} $$

where $T_0^n$ is the forward kinematics of our robot, and $GHM$ is the general homogeneous matrix representing orientation as Euler angles.

When $n \lt 6$, we need to choose for which directions are we planning to solve the inverse kinematics for. We can select maximum $n$ directions. When $n \ge 6$, we can select all six possible directions to solve the inverse kinematics for.

Once we have selected a $min(6,n)$ set of equations of the form:

$$ Z = f(q) $$

we try to invert the equations in the form of:

$$ q = g(Z) $$

by observing if:

* There are equations that involve only one joint coordinate
* There are pairs of equations that can eliminate joint coordinates by division
* There are pairs of equations that can be simplified using trigonometric identities

Once we have identified a solution for one joint coordinate in the following form:

$$ q_i = g_i(Z) $$

then we can use this joint coordinate in following conditions, since its value can be derived solely by the target pose:

$$ q_j = g_j(Z, q_i) = g_j(Z, g_i(Z)) $$

if we exhaust all possibilities with all the pairs of equations, and we still haven't resulted in equations for all the joint coordinates, we can try to multiply both parts from the left of the equation with the inverse transformation matrix of the first kinematic joint. Alternatively, we can multiply from the right both parts of the equation with the inverse transformation of the last kinematic joint.

$$ (T_0^1)^{-1}T_0^n = (T_0^1)^{-1}GHM $$

or

$$ T_0^n(T_{n-1}^n)^{-1} = GHM(T_{n-1}^n)^{-1} $$

this might result in simplifications in the equations and might decouple some pairs. We can apply this method multiple times, e.g.,:

$$ T_0^n(T_{n-1}^n)^{-1}(T_{n-2}^{n-1})^{-1} = GHM(T_{n-1}^n)^{-1}(T_{n-2}^{n-1})^{-1} $$

#### Example with a 2 joint planar robot

Let's calculate the inverse kinematics of a 2 joint planar robot, like the one presented in the figure below

<img src="../artwork/IKM/2_joint_planar_robot.png" width="30%" />

the forward kinematics of this robot arm, equated to the general homogeneous matrix, is given in the following equation:

$$ T_0^3(q_1,q_2) = \begin{bmatrix}
    c_{1,2} & -s_{1,2} & 0 & l_2c_{1,2}+l_1c_1 \\
    s_{1,2} &  c_{1,2} & 0 & l_2s_{1,2}+l_1s_1 \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1
\end{bmatrix} = \begin{bmatrix}
  c_{\theta}c_{\psi} & c_{\psi}s_{\theta}s_{\phi} - c_{\phi}s_{\psi} & s_{\phi}s_{\psi} + c_{\phi}c_{\psi}s_{\theta} & P_x\\
  c_{\theta}s_{\psi} & c_{\phi}c_{\psi} + s_{\theta}s_{\phi}s_{\psi} & c_{\phi}s_{\theta}s_{\psi} - c_{\psi}s_{\phi} & P_y \\
  -s_{\theta} & c_{\theta}s_{\phi}  & c_{\theta}c_{\phi} & P_z \\
  0 & 0 & 0 & 1
  \end{bmatrix}$$

Since this robot has two joints, we can pick $min(6,2) = 2$ equations for calculating the inverse kinematics. For this example, we will pick the $P_x$ and $P_y$ directions.

$$ l_2 cos(q_1+q_2) + l_1 cos q_1 = P_x $$
$$ l_2 sin(q_1+q_2) + l_1 sin q_1 = P_y $$

We can demonstrate that:

$$ P_x^2 + P_y^2 = l_1^2 + l_2^2 + 2l_1 l_2 (c_1 c_{1,2} + s_1 s_{1,2}) $$
$$ P_x^2 + P_y^2 = l_1^2 + l_2^2 + 2l_1 l_2 c_2 $$

and therefore,

$$ q_2 = cos^{-1} \dfrac{P_x^2 + P_y^2 - l_1^2 - l_2^2}{2l_1 l_2} $$

this is the first equation of our inverse kinematics, which provides the necessary joint coordinate $q_2$ for achieving any pose described by $P_x$ and $P_y$ coordinates. For the calculation of the joint coordinate $q_1$ please consult the [lecture slides](https://gitlab.utcluj.ro/rsc/course/-/blob/master/lectures/lecture_04.pdf).

### Geometric solutions

Similarly to analytic solutions, geometric solutions are exact. They are based on geometric relationships between the end-effector, the links lengths, and joint coordinates. In the example below, we demonstrate how to calculate the inverse kinematics for $q_2$ for the same 2 joint planar robot. Considering the figure below:

<img src="../artwork/IKM/cosine_law.png" width=40% />

from Pythagorean theorem, we can calculate the length of the hypotenuse of the red triangle as:

$$ r^2 = P_x^2 + P_y^2 $$

from the law of cosine, we also know that angle $\alpha$ can be calculated as:

$$ cos(\alpha) = \dfrac{l_1^2 + l_2^2 -r^2 }{2 l_1 l_2} = \dfrac{l_1^2 + l_2^2 - P_x^2 - P_y^2 }{2 l_1 l_2} $$

the angles $\alpha$ and $q_2$ are complementary, therefore we can calculate:

$$ q_2 = cos^{-1} \dfrac{P_x^2 + P_y^2 - l_1^2 - l_2^2}{2 l_1 l_2} $$

which is the same solution that we calculated through the analytical method.

### Numerical solutions

Sometimes, the equations involved in the forward kinematics are very complicated and is therefore very difficult, or computationally costly to calculate analytical solutions. In these cases, it might be preferential to seek numerical solutions for the inverse kinematics. As with the other methods, we start by choosing which degrees of freedom are we going to solve the inverse kinematics for. Then we identify the equations involved. Using the example from the previous methodologies, let's assume we have the following equations:

$$ l_2 cos(q_1+q_2) + l_1 cos q_1 = P_x $$
$$ l_2 sin(q_1+q_2) + l_1 sin q_1 = P_y $$

A numerical methodology, would seek values for the joint coordinates $q_1, q_2$ that would minimize the error between the calculated and target $P_x, P_y$. This can be done using several iterative methods, such as Newton-Raphson, Levenberg-Marquardt, or any optimization routine. These routine run over several iterations and they stop either when a maximum number of iterations is exceeded (in which case no valid solution is found), or when an accuracy threshold is reached (in which case the solution is deemed close enough to the target to be considered as the correct one).

## Robotics toolbox
In a previous laboratory, we saw how to define links and how to combine them into a robotic structure, using the __Revolute__, __Prismatic__ and __DHRobot__ methods. We also saw how to calculate the end effector pose using the __fkine__, the jacobian of the robot using the __jacob0__, and how to visualise the robot using the __plot__ methods.

In the examples above, it is 'easy' to calculate the inverse kinematics models by hand, but for more complex robots, we need to solve it numerically. The robotic toolbox has methods for solving the inverse kinematics model, using numerical methods.

The toolbox can do this using the __ikine__ method (from inverse kinematics). The method works by providing a desired end-effector pose (position and orientation) and gives back the joint coordinates the result in the desired pose. As we know, the inverse kinematics model might have more than just one solution for a specific pose. The way numerical methods work, they start looking for a solution around a initial guess (which we can provide), and slowly converge to the joint coordinates that result in the desired pose. The convergence point might depend on the initial guess we are providing.

The __ikine_LM__ method provides a vector of joint coordinates that results in the end-effector pose to be the one desired. To do so, we need to provide a pose as an input to the method. The pose is provided in the form of a 4x4 homogeneous transformation matrix. This transformation matrix provides information about the 6 degrees of freedom available (3 positions and 3 orientations).

The __ikine_LM__ method can solve the inverse kinematics problem even for kinematic chains with less than 6 degrees of freedom. In that case, we need to specify for which DoFs we want the inverse kinematics problem solved. We do this by providing a __mask__ as an input, which is a vector with six logical elements (0 or 1), specifying with 1 those DoFs that we want to solve the inverse kinematics for. The number of DoFs that we can solve the inverse kinematics for __must__ be equal to the DoFs of the robot itself.

## Example

Below, we see an example of how the __ikine_LM__ method works. 

* We first create a DHRobot which represents a robot with two degrees of freedom. 
* Then we create a valid pose (__T_dummy__) by setting the joint coordinates of the robot to a specific value and calculating the forward kinematics.
* Then we try to solve the inverse kinematics for that specific pose, and we obtain the joint coordinates (__q_comp__) that would result in that specific pose.
* We verify that the inverse kinematics give the same solution as the input to the forward kinematics
* We can also try to solve the inverse kinematics for any input pose (example 5.2.2)

For the input pose to be valid, we need to use the __SE3__ constructors from the spatial math module of the robotics toolbox. The corresponding methods are:

* Translation: SE3(x,y,z)
* Rotation on X: SE3.Rx(phi)
* Rotation on Y: SE3.Ry(theta)
* Rotation on Z: SE3.Rz(psi)

In [5]:
# 5.2.1
# example of use-case
from roboticstoolbox import *
from spatialmath import *
import numpy as np

# create the robot
#  theta, d, r , alpha, offset, qlim, mdh
Link1 = RevoluteMDH(d = 0.3);
Link2 = RevoluteMDH(alpha = np.pi/2)
Link3 = RevoluteMDH(a = 0.3)

# we combine the links using DHRobot
rob = DHRobot([
    Link1,
    Link2,
    Link3],  name = "my_robot", tool=SE3(2,0,0))
q_dummy  = np.array([0, np.pi/2, -np.pi/4])

# test the ikine 
T  = rob.fkine(q_dummy)
print("Desired pose: \n", T)
sol_comp = rob.ikine_LM(T, mask=np.array([1,1,1,0,0,0]))
print('Joint coordinates that we used for generating the input pose:', q_dummy)
print('Joint coordinates computed by inverse kinematics for the input pose:', sol_comp.q)

#you can also input an initial guess q0
sol_comp2 = rob.ikine_LM(T, q0=q_dummy+0.2, mask=np.array([1,1,1,0,0,0]))
print('Joint coordinates computed by inverse kinematics for the input pose, with initial guess:', sol_comp2.q)

Desired pose: 
    0.7071   -0.7071    0         1.414     
   0         0        -1         0         
   0.7071    0.7071    0         2.014     
   0         0         0         1         

Joint coordinates that we used for generating the input pose: [ 0.          1.57079633 -0.78539816]
Joint coordinates computed by inverse kinematics for the input pose: [-4.63814105e-30  1.91205003e-01  7.85398163e-01]
Joint coordinates computed by inverse kinematics for the input pose, with initial guess: [-1.05565852e-28  1.57079633e+00 -7.85398164e-01]


In [6]:
# 5.2.2
# example of use-case
from spatialmath import *

# We can also try to solve the inverse kinematics for any arbitrary pose
# Try to play with the mask, to see that each time the solution is different

T = SE3(0.2, 0.35, 2.3)
sol = rob.ikine_LM(T, mask=np.array([1,1,1,0,0,0]))
print('The joint coordinates that take us to pose T are: ', sol.q)
print('The desired pose was: \n', T, 'while the actual pose is: \n', rob.fkine(sol.q))

The joint coordinates that take us to pose T are:  [1.05165021 0.00886826 1.51034284]
The desired pose was: 
    1         0         0         0.2       
   0         1         0         0.35      
   0         0         1         2.3       
   0         0         0         1         
 while the actual pose is: 
    0.02558  -0.4955    0.8682    0.2       
   0.04477  -0.8671   -0.4961    0.35      
   0.9987    0.05156   0         2.3       
   0         0         0         1         



AttributeError: module 'numpy' has no attribute 'cotan'